# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    vocab = sorted(set(text))
    vocab_to_int = {c: i for i, c in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))
    
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    d ={".":"||Period||", ",":"||Comma||", 
        '"': "||Quotation_Mark", "!": "||Exclamation_Mark||",
       "?": "||Question_Mark||", "(": "||Left_Parentheses||",
       ")": "||Right_Parentheses||", "--": "||Dash||",
       "\n": "||Return||", ";": "||Semicolon||"}
    return d

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [8]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, [None,None], name='input')
    targets = tf.placeholder(tf.int32, [None,None], name="targets")
    
    lr = tf.placeholder(tf.float32, name='lr')
    return inputs, targets, lr


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [9]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    def build_cell(rnn_size):
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        return lstm
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size) for _ in range(3)])
    initial_state = tf.identity(cell.zero_state(batch_size, tf.float32), name='initial_state')
    return cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [10]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    word_embedding = tf.get_variable("word_embedding", 
                                     [vocab_size, embed_dim])
    embedded_word_ids = tf.nn.embedding_lookup(word_embedding, input_data )
    return embedded_word_ids


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [11]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    output, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state,name='final_state')
    
    return output, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [12]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embedding = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embedding)
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None, weights_initializer=tf.truncated_normal_initializer(stddev= 0.1),biases_initializer=tf.zeros_initializer())
    
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [13]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    chars_per_batch = batch_size * seq_length
    n_batches = len(int_text)//chars_per_batch
    
    
    input_data = np.array(int_text[:(n_batches * chars_per_batch)])
    target_data = np.array(int_text[1 : (n_batches * chars_per_batch)+1])
    target_data[-1] = input_data[0]
    
    inputs = input_data.reshape(batch_size, -1)
    targets = target_data.reshape(batch_size, -1)
   
    inputs = np.split(inputs, n_batches, -1)
    targets = np.split(targets, n_batches, -1)
    
    batches = np.array(list(zip(inputs, targets)))
    return batches

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [14]:
# Number of Epochs
num_epochs = 200
# Batch Size
batch_size = 50
# RNN Size
rnn_size = 300
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 20
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 10

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/69   train_loss = 8.822
Epoch   0 Batch   10/69   train_loss = 6.488
Epoch   0 Batch   20/69   train_loss = 6.312
Epoch   0 Batch   30/69   train_loss = 6.240
Epoch   0 Batch   40/69   train_loss = 6.088
Epoch   0 Batch   50/69   train_loss = 6.073
Epoch   0 Batch   60/69   train_loss = 5.983
Epoch   1 Batch    1/69   train_loss = 6.018
Epoch   1 Batch   11/69   train_loss = 6.020
Epoch   1 Batch   21/69   train_loss = 5.742
Epoch   1 Batch   31/69   train_loss = 5.610
Epoch   1 Batch   41/69   train_loss = 5.766
Epoch   1 Batch   51/69   train_loss = 5.671
Epoch   1 Batch   61/69   train_loss = 5.433
Epoch   2 Batch    2/69   train_loss = 5.412
Epoch   2 Batch   12/69   train_loss = 5.225
Epoch   2 Batch   22/69   train_loss = 5.227
Epoch   2 Batch   32/69   train_loss = 5.099
Epoch   2 Batch   42/69   train_loss = 5.283
Epoch   2 Batch   52/69   train_loss = 5.088
Epoch   2 Batch   62/69   train_loss = 5.082
Epoch   3 Batch    3/69   train_loss = 4.929
Epoch   3 

Epoch  26 Batch   36/69   train_loss = 3.586
Epoch  26 Batch   46/69   train_loss = 3.627
Epoch  26 Batch   56/69   train_loss = 3.693
Epoch  26 Batch   66/69   train_loss = 3.632
Epoch  27 Batch    7/69   train_loss = 3.575
Epoch  27 Batch   17/69   train_loss = 3.515
Epoch  27 Batch   27/69   train_loss = 3.644
Epoch  27 Batch   37/69   train_loss = 3.722
Epoch  27 Batch   47/69   train_loss = 3.709
Epoch  27 Batch   57/69   train_loss = 3.659
Epoch  27 Batch   67/69   train_loss = 3.457
Epoch  28 Batch    8/69   train_loss = 3.702
Epoch  28 Batch   18/69   train_loss = 3.656
Epoch  28 Batch   28/69   train_loss = 3.748
Epoch  28 Batch   38/69   train_loss = 3.587
Epoch  28 Batch   48/69   train_loss = 3.450
Epoch  28 Batch   58/69   train_loss = 3.632
Epoch  28 Batch   68/69   train_loss = 3.588
Epoch  29 Batch    9/69   train_loss = 3.575
Epoch  29 Batch   19/69   train_loss = 3.587
Epoch  29 Batch   29/69   train_loss = 3.631
Epoch  29 Batch   39/69   train_loss = 3.649
Epoch  29 

Epoch  53 Batch    3/69   train_loss = 2.425
Epoch  53 Batch   13/69   train_loss = 2.474
Epoch  53 Batch   23/69   train_loss = 2.501
Epoch  53 Batch   33/69   train_loss = 2.497
Epoch  53 Batch   43/69   train_loss = 2.411
Epoch  53 Batch   53/69   train_loss = 2.515
Epoch  53 Batch   63/69   train_loss = 2.629
Epoch  54 Batch    4/69   train_loss = 2.358
Epoch  54 Batch   14/69   train_loss = 2.428
Epoch  54 Batch   24/69   train_loss = 2.451
Epoch  54 Batch   34/69   train_loss = 2.531
Epoch  54 Batch   44/69   train_loss = 2.672
Epoch  54 Batch   54/69   train_loss = 2.405
Epoch  54 Batch   64/69   train_loss = 2.445
Epoch  55 Batch    5/69   train_loss = 2.507
Epoch  55 Batch   15/69   train_loss = 2.466
Epoch  55 Batch   25/69   train_loss = 2.442
Epoch  55 Batch   35/69   train_loss = 2.591
Epoch  55 Batch   45/69   train_loss = 2.667
Epoch  55 Batch   55/69   train_loss = 2.614
Epoch  55 Batch   65/69   train_loss = 2.314
Epoch  56 Batch    6/69   train_loss = 2.533
Epoch  56 

Epoch  79 Batch   39/69   train_loss = 1.724
Epoch  79 Batch   49/69   train_loss = 1.803
Epoch  79 Batch   59/69   train_loss = 1.701
Epoch  80 Batch    0/69   train_loss = 1.777
Epoch  80 Batch   10/69   train_loss = 1.732
Epoch  80 Batch   20/69   train_loss = 1.593
Epoch  80 Batch   30/69   train_loss = 1.737
Epoch  80 Batch   40/69   train_loss = 1.619
Epoch  80 Batch   50/69   train_loss = 1.791
Epoch  80 Batch   60/69   train_loss = 1.628
Epoch  81 Batch    1/69   train_loss = 1.732
Epoch  81 Batch   11/69   train_loss = 1.856
Epoch  81 Batch   21/69   train_loss = 1.848
Epoch  81 Batch   31/69   train_loss = 1.723
Epoch  81 Batch   41/69   train_loss = 1.633
Epoch  81 Batch   51/69   train_loss = 1.758
Epoch  81 Batch   61/69   train_loss = 1.694
Epoch  82 Batch    2/69   train_loss = 1.741
Epoch  82 Batch   12/69   train_loss = 1.730
Epoch  82 Batch   22/69   train_loss = 1.669
Epoch  82 Batch   32/69   train_loss = 1.789
Epoch  82 Batch   42/69   train_loss = 1.711
Epoch  82 

Epoch 106 Batch    6/69   train_loss = 1.465
Epoch 106 Batch   16/69   train_loss = 1.371
Epoch 106 Batch   26/69   train_loss = 1.389
Epoch 106 Batch   36/69   train_loss = 1.344
Epoch 106 Batch   46/69   train_loss = 1.345
Epoch 106 Batch   56/69   train_loss = 1.302
Epoch 106 Batch   66/69   train_loss = 1.458
Epoch 107 Batch    7/69   train_loss = 1.472
Epoch 107 Batch   17/69   train_loss = 1.351
Epoch 107 Batch   27/69   train_loss = 1.343
Epoch 107 Batch   37/69   train_loss = 1.335
Epoch 107 Batch   47/69   train_loss = 1.351
Epoch 107 Batch   57/69   train_loss = 1.336
Epoch 107 Batch   67/69   train_loss = 1.302
Epoch 108 Batch    8/69   train_loss = 1.399
Epoch 108 Batch   18/69   train_loss = 1.282
Epoch 108 Batch   28/69   train_loss = 1.402
Epoch 108 Batch   38/69   train_loss = 1.302
Epoch 108 Batch   48/69   train_loss = 1.284
Epoch 108 Batch   58/69   train_loss = 1.294
Epoch 108 Batch   68/69   train_loss = 1.312
Epoch 109 Batch    9/69   train_loss = 1.334
Epoch 109 

Epoch 132 Batch   42/69   train_loss = 1.120
Epoch 132 Batch   52/69   train_loss = 1.173
Epoch 132 Batch   62/69   train_loss = 1.069
Epoch 133 Batch    3/69   train_loss = 1.156
Epoch 133 Batch   13/69   train_loss = 1.139
Epoch 133 Batch   23/69   train_loss = 1.066
Epoch 133 Batch   33/69   train_loss = 1.174
Epoch 133 Batch   43/69   train_loss = 0.994
Epoch 133 Batch   53/69   train_loss = 1.061
Epoch 133 Batch   63/69   train_loss = 1.064
Epoch 134 Batch    4/69   train_loss = 1.077
Epoch 134 Batch   14/69   train_loss = 1.110
Epoch 134 Batch   24/69   train_loss = 1.086
Epoch 134 Batch   34/69   train_loss = 1.204
Epoch 134 Batch   44/69   train_loss = 1.170
Epoch 134 Batch   54/69   train_loss = 1.075
Epoch 134 Batch   64/69   train_loss = 1.117
Epoch 135 Batch    5/69   train_loss = 1.115
Epoch 135 Batch   15/69   train_loss = 1.124
Epoch 135 Batch   25/69   train_loss = 1.041
Epoch 135 Batch   35/69   train_loss = 1.101
Epoch 135 Batch   45/69   train_loss = 1.079
Epoch 135 

Epoch 159 Batch    9/69   train_loss = 0.975
Epoch 159 Batch   19/69   train_loss = 0.994
Epoch 159 Batch   29/69   train_loss = 0.953
Epoch 159 Batch   39/69   train_loss = 1.073
Epoch 159 Batch   49/69   train_loss = 1.033
Epoch 159 Batch   59/69   train_loss = 0.933
Epoch 160 Batch    0/69   train_loss = 0.972
Epoch 160 Batch   10/69   train_loss = 0.999
Epoch 160 Batch   20/69   train_loss = 0.976
Epoch 160 Batch   30/69   train_loss = 0.962
Epoch 160 Batch   40/69   train_loss = 0.987
Epoch 160 Batch   50/69   train_loss = 1.109
Epoch 160 Batch   60/69   train_loss = 1.041
Epoch 161 Batch    1/69   train_loss = 1.018
Epoch 161 Batch   11/69   train_loss = 1.030
Epoch 161 Batch   21/69   train_loss = 1.093
Epoch 161 Batch   31/69   train_loss = 1.039
Epoch 161 Batch   41/69   train_loss = 1.040
Epoch 161 Batch   51/69   train_loss = 1.012
Epoch 161 Batch   61/69   train_loss = 1.016
Epoch 162 Batch    2/69   train_loss = 0.995
Epoch 162 Batch   12/69   train_loss = 0.950
Epoch 162 

Epoch 185 Batch   45/69   train_loss = 0.855
Epoch 185 Batch   55/69   train_loss = 0.856
Epoch 185 Batch   65/69   train_loss = 0.824
Epoch 186 Batch    6/69   train_loss = 0.952
Epoch 186 Batch   16/69   train_loss = 0.863
Epoch 186 Batch   26/69   train_loss = 0.946
Epoch 186 Batch   36/69   train_loss = 0.868
Epoch 186 Batch   46/69   train_loss = 0.829
Epoch 186 Batch   56/69   train_loss = 0.854
Epoch 186 Batch   66/69   train_loss = 0.865
Epoch 187 Batch    7/69   train_loss = 0.904
Epoch 187 Batch   17/69   train_loss = 0.769
Epoch 187 Batch   27/69   train_loss = 0.863
Epoch 187 Batch   37/69   train_loss = 0.850
Epoch 187 Batch   47/69   train_loss = 0.938
Epoch 187 Batch   57/69   train_loss = 0.781
Epoch 187 Batch   67/69   train_loss = 0.879
Epoch 188 Batch    8/69   train_loss = 0.815
Epoch 188 Batch   18/69   train_loss = 0.788
Epoch 188 Batch   28/69   train_loss = 0.821
Epoch 188 Batch   38/69   train_loss = 0.870
Epoch 188 Batch   48/69   train_loss = 0.748
Epoch 188 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [19]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    
    with loaded_graph.as_default():
        input_tensor = loaded_graph.get_tensor_by_name("input:0")
        initial_state = loaded_graph.get_tensor_by_name("initial_state:0")
        final_state = loaded_graph.get_tensor_by_name("final_state:0")
        probs = loaded_graph.get_tensor_by_name("probs:0")
    return input_tensor, initial_state, final_state, probs


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [20]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    rand_word = np.random.choice(list(int_to_vocab.values()), 1, p=probabilities)[0]
    
    return rand_word


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [21]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak: eh, gee, dad. all of the black where i don't know, say. i'm the great product to find out with worse. you gotta use my money is i need a beer to save the world rather have to buy that.
moe_szyslak: i need all willy / like the worst next on / no more car!
moe_szyslak: eh, big place knows that, chief with lenny. / oh who i probably need back to the mechanical guy.
lenny_leonard: but where was i bought me with all the ladies guest." oh moe, didn't you like is my friends! i'll. you want to focus with three days. wasn't a one-hour need in this place now. you gotta take an own pal!
lloyd: take this joint cards? i'll take you guys, got a good friend?
moe_szyslak: yeah, three big moron and make sure. but as it says of this drink. i'll mean it all.
homer_simpson:(computer voice) you got the tv of my house when he got the computer thing


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.